In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import yaml
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import VotingClassifier

from src.data.prepare_data import prepare_data
from src.models.utils import imbalanced_sampling
from src.models.model import Classifier
from src.models.evaluation import Evaluation

In [2]:
# read config
with open('../'+'config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
df = pd.read_csv('../'+config['data_loader']['path'])
df = prepare_data(df=df)
display(df.head())

,time,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v21,v22,v23,v24,v25,v26,v27,v28,amount,label
0,-1.996580,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,-1.996580,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342474,0
2,-1.996558,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160684,0
3,-1.996558,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,-1.996537,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


In [4]:
# split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,:-1], df['label']
    , test_size=config['train_test_split']['test_size']
    , random_state=42
    , shuffle=True
    , stratify=df['label']
    )

In [5]:
# apply oversampling to the train set
X_train_rs, y_train_rs = imbalanced_sampling(
    method="over"
    , X_train=X_train
    , y_train=y_train
)

In [6]:
lgbm_params = {
    "colsample_bytree": 0.40921304830970556, "learning_rate": 0.3686770314875885, "max_depth": 20, "num_leaves": 78, "n_estimators": 140, "reg_alpha": 1.6443457513284063, "reg_lambda": 0.6750277604651747,
    "boosting_type": "gbdt", "subsample_for_bin": 200000, "objective": "binary", "random_state": 42
    }
rf_params = {
    "max_depth": 20, "max_features": 9, "min_samples_leaf": 3, "min_samples_split": 9, "n_estimators": 5,
    "criterion": "gini", "bootstrap": True, "oob_score": False, "verbose": 0, "random_state": 42
    }
dt_params = {
    "max_depth": 17, "min_samples_leaf": 6,
    "criterion": "gini", "splitter": "best", "random_state": 42
    }

In [7]:
# build ensemble
clf = VotingClassifier(
    estimators = [
        ('LGBMClassifier', Classifier(algorithm='LGBMClassifier', **lgbm_params).model)
        , ('RandomForestClassifier', Classifier(algorithm='RandomForestClassifier', **rf_params).model)
        , ('DecisionTreeClassifier', Classifier(algorithm='DecisionTreeClassifier', **dt_params).model)
    ]
    , voting='soft'
)
clf.fit(X_train_rs, y_train_rs)

[LightGBM] [Info] Number of positive: 270099, number of negative: 270099
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002394 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 540198, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

VotingClassifier(estimators=[('LGBMClassifier',
                              LGBMClassifier(colsample_bytree=0.40921304830970556,
                                             learning_rate=0.3686770314875885,
                                             max_depth=20, n_estimators=140,
                                             num_leaves=78, objective='binary',
                                             random_state=42,
                                             reg_alpha=1.6443457513284063,
                                             reg_lambda=0.6750277604651747)),
                             ('RandomForestClassifier',
                              RandomForestClassifier(max_depth=20,
                                                     max_features=9,
                                                     min_samples_leaf=3,
                                                     min_samples_split=9,
                                                     n_estimators=5,
                                                     random_state=42)),
                             ('DecisionTreeClassifier',
                              DecisionTreeClassifier(max_depth=17,
                                                     min_samples_leaf=6,
                                                     random_state=42))],
                 voting='soft')

In [8]:
# check individual and overall accuracy
for name, model in clf.named_estimators_.items():
    print(f'{name}: {model.score(X_test, y_test)}')

clf.score(X_test, y_test)

LGBMClassifier: 0.9994382416965101
RandomForestClassifier: 0.9992978021206376
DecisionTreeClassifier: 0.9972614282704866


0.9995084614844463